<a href="https://colab.research.google.com/github/Abdur-roheem/machine-learning-zoomcamp-homework/blob/main/homework8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip


--2025-12-08 06:58:52--  https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
Resolving github.com (github.com)... 140.82.114.3
Connecting to github.com (github.com)|140.82.114.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/405934815/e712cf72-f851-44e0-9c05-e711624af985?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-12-08T07%3A46%3A52Z&rscd=attachment%3B+filename%3Ddata.zip&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-12-08T06%3A45%3A55Z&ske=2025-12-08T07%3A46%3A52Z&sks=b&skv=2018-11-09&sig=Ad7X5VW4m0kTaiLqsSDtmec5iZZPkzYR10v38MA2m2c%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2NTE3ODkzMiwibmJmIjoxNzY1MTc3MTMyLCJwYXRoIjoicmVsZWFzZWFzc2V0cHJvZHVjdGlvbi5ibG

In [2]:
!unzip data.zip

Archive:  data.zip
replace data/test/curly/03312ac556a7d003f7570657f80392c34.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms

In [4]:

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [5]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [6]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()

        self.conv_layer = nn.Sequential(
            nn.Conv2d(
                in_channels=3,
                out_channels=32,
                kernel_size=3,
                stride=1,
                padding=0
            ),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # Calculate size after conv + pool:
        # Input: 200x200 → Conv(kernel=3) → 198x198 → MaxPool(2) → 99x99
        conv_output_size = 32 * 99 * 99

        # Fully connected layers

        self.fc_layers = nn.Sequential(
            nn.Flatten(),
            nn.Linear(conv_output_size, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        x = self.conv_layer(x)
        x = self.fc_layers(x)
        return x


# Instantiate model
model = SimpleCNN()
model.to(device)

# Example check
print(model)


SimpleCNN(
  (conv_layer): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc_layers): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=313632, out_features=64, bias=True)
    (2): ReLU()
    (3): Linear(in_features=64, out_features=1, bias=True)
  )
)


nn.BCEWithLogitsLoss()

In [7]:
# Option 1: Using torchsummary (install with: pip install torchsummary)
from torchsummary import summary
summary(model, input_size=(3, 200, 200))

# Option 2: Manual counting
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 32, 198, 198]             896
              ReLU-2         [-1, 32, 198, 198]               0
         MaxPool2d-3           [-1, 32, 99, 99]               0
           Flatten-4               [-1, 313632]               0
            Linear-5                   [-1, 64]      20,072,512
              ReLU-6                   [-1, 64]               0
            Linear-7                    [-1, 1]              65
Total params: 20,073,473
Trainable params: 20,073,473
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.46
Forward/backward pass size (MB): 23.93
Params size (MB): 76.57
Estimated Total Size (MB): 100.96
----------------------------------------------------------------
Total parameters: 20073473


20073473

In [8]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    ) # ImageNet normalization
])

In [9]:
from torch.utils.data import Dataset
from torchvision.datasets import ImageFolder
import os

class ClothingDataset(Dataset):
    def __init__(self, data_dir, transform=None):
        self.data_dir = data_dir
        self.transform = transform
        self.image_folder = ImageFolder(data_dir, transform=transform)

    def __len__(self):
        return len(self.image_folder)

    def __getitem__(self, idx):
        return self.image_folder[idx]

In [10]:
from torch.utils.data import DataLoader

train_dataset = ClothingDataset(
    data_dir='/content/data/train',
    transform=train_transforms
)

validation_dataset = ClothingDataset(
    data_dir='/content/data/test',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=20, shuffle=False)

In [11]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()

In [12]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

# ---------------- Validation -------------------
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6665, Acc: 0.6112, Val Loss: 0.6511, Val Acc: 0.6617
Epoch 2/10, Loss: 0.5702, Acc: 0.6787, Val Loss: 0.6332, Val Acc: 0.6318
Epoch 3/10, Loss: 0.5207, Acc: 0.7350, Val Loss: 0.6143, Val Acc: 0.6766
Epoch 4/10, Loss: 0.4773, Acc: 0.7600, Val Loss: 0.6049, Val Acc: 0.6617
Epoch 5/10, Loss: 0.4606, Acc: 0.7550, Val Loss: 0.7307, Val Acc: 0.5672
Epoch 6/10, Loss: 0.3954, Acc: 0.8275, Val Loss: 0.6412, Val Acc: 0.6866
Epoch 7/10, Loss: 0.2844, Acc: 0.8838, Val Loss: 0.8307, Val Acc: 0.6816
Epoch 8/10, Loss: 0.2885, Acc: 0.8788, Val Loss: 0.7052, Val Acc: 0.7114
Epoch 9/10, Loss: 0.1882, Acc: 0.9313, Val Loss: 0.9275, Val Acc: 0.6866
Epoch 10/10, Loss: 0.2585, Acc: 0.8912, Val Loss: 0.8158, Val Acc: 0.6915


In [13]:
history

{'acc': [0.61125,
  0.67875,
  0.735,
  0.76,
  0.755,
  0.8275,
  0.88375,
  0.87875,
  0.93125,
  0.89125],
 'loss': [0.6664755456149578,
  0.5701714769005776,
  0.5206970952451229,
  0.47728381007909776,
  0.4605835236608982,
  0.39537646248936653,
  0.2844009276479483,
  0.28851776048541067,
  0.18818021453917028,
  0.2584627663716674],
 'val_acc': [0.6616915422885572,
  0.6318407960199005,
  0.6766169154228856,
  0.6616915422885572,
  0.5671641791044776,
  0.6865671641791045,
  0.681592039800995,
  0.7114427860696517,
  0.6865671641791045,
  0.6915422885572139],
 'val_loss': [0.6510908523602272,
  0.6332464968387167,
  0.6142638984011181,
  0.6048766149217216,
  0.730717788139979,
  0.641188201026537,
  0.8306739904097656,
  0.70519710061562,
  0.9274571704775539,
  0.8157966108761023]}

In [14]:
hist = {'acc': [0.61125,
  0.67875,
  0.735,
  0.76,
  0.755,
  0.8275,
  0.88375,
  0.87875,
  0.93125,
  0.89125],
 'loss': [0.6664755456149578,
  0.5701714769005776,
  0.5206970952451229,
  0.47728381007909776,
  0.4605835236608982,
  0.39537646248936653,
  0.2844009276479483,
  0.28851776048541067,
  0.18818021453917028,
  0.2584627663716674],
 'val_acc': [0.6616915422885572,
  0.6318407960199005,
  0.6766169154228856,
  0.6616915422885572,
  0.5671641791044776,
  0.6865671641791045,
  0.681592039800995,
  0.7114427860696517,
  0.6865671641791045,
  0.6915422885572139],
 'val_loss': [0.6510908523602272,
  0.6332464968387167,
  0.6142638984011181,
  0.6048766149217216,
  0.730717788139979,
  0.641188201026537,
  0.8306739904097656,
  0.70519710061562,
  0.9274571704775539,
  0.8157966108761023]}

In [15]:
import pandas as pd

history = pd.DataFrame(history)


In [16]:
history

,acc,loss,val_acc,val_loss
0,0.61125,0.666476,0.661692,0.651091
1,0.67875,0.570171,0.631841,0.633246
2,0.73500,0.520697,0.676617,0.614264
3,0.76000,0.477284,0.661692,0.604877
4,0.75500,0.460584,0.567164,0.730718
5,0.82750,0.395376,0.686567,0.641188
6,0.88375,0.284401,0.681592,0.830674
7,0.87875,0.288518,0.711443,0.705197
8,0.93125,0.188180,0.686567,0.927457
9,0.89125,0.258463,0.691542,0.815797


In [17]:
history.median()

,0
acc,0.793750
loss,0.427980
val_acc,0.679104
val_loss,0.678144


In [18]:
history.std()

,0
acc,0.104071
loss,0.154085
val_acc,0.040676
val_loss,0.109296


In [19]:
transforms.RandomRotation(50),
transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
transforms.RandomHorizontalFlip(),

RandomHorizontalFlip(p=0.5)

In [19]:
train_transforms = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

In [20]:
from torch.utils.data import DataLoader

train_dataset = ClothingDataset(
    data_dir='/content/data/train',
    transform=train_transforms
)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True)

In [21]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1) # Ensure labels are float and have shape (batch_size, 1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        # For binary classification with BCEWithLogitsLoss, apply sigmoid to outputs before thresholding for accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

# ---------------- Validation -------------------
    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in validation_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(validation_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6307, Acc: 0.6600, Val Loss: 0.5963, Val Acc: 0.7114
Epoch 2/10, Loss: 0.5801, Acc: 0.6750, Val Loss: 0.5968, Val Acc: 0.7214
Epoch 3/10, Loss: 0.5374, Acc: 0.7175, Val Loss: 0.5870, Val Acc: 0.7015
Epoch 4/10, Loss: 0.5502, Acc: 0.7125, Val Loss: 0.5695, Val Acc: 0.7264
Epoch 5/10, Loss: 0.5250, Acc: 0.7412, Val Loss: 0.6758, Val Acc: 0.6816
Epoch 6/10, Loss: 0.5141, Acc: 0.7612, Val Loss: 0.5764, Val Acc: 0.7313
Epoch 7/10, Loss: 0.5217, Acc: 0.7362, Val Loss: 0.5756, Val Acc: 0.6866
Epoch 8/10, Loss: 0.4888, Acc: 0.7588, Val Loss: 0.5724, Val Acc: 0.7214
Epoch 9/10, Loss: 0.4989, Acc: 0.7475, Val Loss: 0.5452, Val Acc: 0.7313
Epoch 10/10, Loss: 0.4663, Acc: 0.7688, Val Loss: 0.5619, Val Acc: 0.7164


In [24]:
history = pd.DataFrame(history)

In [27]:
history.mean()

,0
acc,0.727875
loss,0.531328
val_acc,0.712935
val_loss,0.585693


In [26]:
history.std()

,0
acc,0.036695
loss,0.047329
val_acc,0.017714
val_loss,0.035250
